In [ ]:
## DCA First Try
# Define the model

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    # Dilated CNN block
    x = Conv2D(64, (3, 3), dilation_rate=dilation_rate, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(128, (3, 3), dilation_rate=dilation_rate, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2, 2))(x)

    # Attention mechanism
    attention = GlobalAveragePooling2D()(x)
    attention = Dense(attention_units, activation='relu')(attention)
    attention = Dense(attention_units, activation='sigmoid')(attention)
    attention = Multiply()([x, attention])

    # Global average pooling and final dense layer
    x = GlobalAveragePooling2D()(attention)
    x = Dense(256, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    return model


In [1]:
## Self Attention

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    # Dilated CNN block
    x = Conv2D(64, (3, 3), dilation_rate=dilation_rate, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(128, (3, 3), dilation_rate=dilation_rate, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2, 2))(x)

    # Attention mechanism
    # attention = GlobalAveragePooling2D()(x)
    attention = Attention()([x, x])
    attention = Dense(attention_units, activation='relu')(attention)
    attention = Dense(attention_units, activation='sigmoid')(attention)
    attention = Multiply()([x, attention])

    # Global average pooling and final dense layer
    x = GlobalAveragePooling2D()(attention)
    x = Dense(256, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    return model

In [ ]:
# Squeeze and Excitation (SE)

def squeeze_excite_block(input_tensor, ratio=8):
    # Squeeze operation
    squeeze = GlobalAveragePooling2D()(input_tensor)

    # Excitation operation
    excitation = Dense(input_tensor.shape[-1] // ratio, activation='relu')(squeeze)
    excitation = Dense(input_tensor.shape[-1], activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, -1))(excitation)

    # Scale the input tensor
    scaled_input = Multiply()([input_tensor, excitation])

    return scaled_input

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    # Dilated CNN block
    x = Conv2D(64, (3, 3), dilation_rate=dilation_rate, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(128, (3, 3), dilation_rate=dilation_rate, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2, 2))(x)

    # Attention mechanism (Squeeze-and-Excite)
    attention = squeeze_excite_block(x, ratio=8)

    # Global average pooling and final dense layer
    x = GlobalAveragePooling2D()(attention)
    x = Dense(256, activation='relu')(x)
    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    return model

In [ ]:
## SE Extra 

# Define the model


def squeeze_excite_block(input_tensor, ratio=8):
    # Squeeze operation
    squeeze = GlobalAveragePooling2D()(input_tensor)

    # Excitation operation
    excitation = Dense(input_tensor.shape[-1] // ratio, activation='relu')(squeeze)
    excitation = Dense(input_tensor.shape[-1], activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, -1))(excitation)

    # Scale the input tensor
    scaled_input = Multiply()([input_tensor, excitation])

    return scaled_input

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    # Dilated CNN block
    x = Conv2D(64, (3, 3), dilation_rate=dilation_rate, padding='same', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = Conv2D(128, (3, 3), dilation_rate=dilation_rate, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2, 2))(x)

    # Attention mechanism (Squeeze-and-Excite)
    attention = squeeze_excite_block(x, ratio=8)

    # Additional Convolutional layers
    x = Conv2D(256, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01))(attention)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    x = MaxPooling2D((2, 2))(x)

    dropout_rate=0.5
    # Global average pooling and final dense layer
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    return model


In [ ]:
# CBAM attention

# Define the model
def convolutional_block_attention_module(input_tensor, ratio=8):
    # Channel attention
    channel_avg = GlobalAveragePooling2D()(input_tensor)
    channel_max = GlobalAveragePooling2D()(input_tensor)
    channel_shared = Dense(input_tensor.shape[-1] // ratio, activation='relu', kernel_regularizer=regularizers.l2(0.01))(channel_avg) + \
                     Dense(input_tensor.shape[-1] // ratio, activation='relu', kernel_regularizer=regularizers.l2(0.01))(channel_max)
    channel_attention = Dense(input_tensor.shape[-1], activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(channel_shared)
    channel_attention = Reshape((1, 1, -1))(channel_attention)

    # Spatial attention
    spatial_avg = Reshape((1, 1, input_tensor.shape[-1]))(GlobalAveragePooling2D()(input_tensor))
    spatial_max = Reshape((1, 1, input_tensor.shape[-1]))(GlobalAveragePooling2D()(input_tensor))
    spatial_shared = Add()([Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(spatial_avg),
                           Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(spatial_max)])
    spatial_attention = Activation('sigmoid')(spatial_shared)

    # Combine channel and spatial attention
    attention = Multiply()([input_tensor, channel_attention, spatial_attention])

    return attention

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    # Dilated CNN block
    x = Conv2D(64, (3, 3), dilation_rate=dilation_rate, padding='same', kernel_regularizer=regularizers.l2(0.01))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Apply CBAM
    # x = convolutional_block_attention_module(x)

    x = Conv2D(128, (3, 3), dilation_rate=dilation_rate, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Apply CBAM
    x = convolutional_block_attention_module(x)

    x = MaxPooling2D((2, 2))(x)

    # Additional Convolutional layers
    x = Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Apply CBAM
    x = convolutional_block_attention_module(x)

    x = MaxPooling2D((2, 2))(x)

    dropout_rate=0.5
    # Global average pooling and final dense layer
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(dropout_rate)(x)

    output_tensor = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=input_tensor, outputs=output_tensor)

    return model


In [ ]:
# Self Attention from SE
# Define the model
def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    input_tensor = Input(shape=input_shape)

    conv1 = Conv2D(filters = 64, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(input_tensor)
    maxpool1 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv1)

    # attn1 = Attention(maxpool1, ))

    conv2 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv2)

    conv3 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool2)
    maxpool3 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv3)

    attn = Attention(attention_units)([maxpool3, maxpool3])

    flattened = Flatten()(attn)

    fc1 = Dense(units = 512, activation = 'relu')(flattened)
    drop1 = Dropout(rate = 0.5)(fc1)

    final = Dense(units = num_classes,activation='sigmoid')(drop1)

    model = Model(inputs=input_tensor, outputs=final)

    return model


In [ ]:
# SE with VGG16

# Define the model
def squeeze_excite_block(input_tensor, ratio=8):
    # Squeeze operation
    squeeze = GlobalAveragePooling2D()(input_tensor)

    # Excitation operation
    excitation = Dense(input_tensor.shape[-1] // ratio, activation='relu')(squeeze)
    excitation = Dense(input_tensor.shape[-1], activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, -1))(excitation)

    # Scale the input tensor
    scaled_input = Multiply()([input_tensor, excitation])

    return scaled_input

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    # Load pre-trained VGG16 model without top (fully connected) layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Get output tensor of the last VGG16 layer
    x = base_model.output

    # Dilated CNN block with attention
    x = Conv2D(256, (3, 3), dilation_rate=dilation_rate, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = squeeze_excite_block(x)

    # Additional Convolutional layers if needed
    x = Conv2D(512, (3, 3), dilation_rate=dilation_rate, padding='same', kernel_regularizer=regularizers.l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = squeeze_excite_block(x)

    # Global average pooling and dense layers
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dense(256, activation='relu')(x)

    # Output layer
    predictions = Dense(num_classes, activation='softmax')(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    return model


In [ ]:
# SA from SE on VGG16

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    # Load pre-trained VGG16 model without top (fully connected) layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Get output tensor of the last VGG16 layer
    x = base_model.output

    conv1 = Conv2D(filters = 64, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(x)
    maxpool1 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv1)

    # attn1 = Attention(maxpool1, ))

    conv2 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv2)

    conv3 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool2)
    maxpool3 = MaxPooling2D(pool_size = (1, 1), strides=(2, 2))(conv3)

    attn = Attention(attention_units)([maxpool3, maxpool3])

    flattened = Flatten()(attn)

    fc1 = Dense(units = 512, activation = 'relu')(flattened)
    drop1 = Dropout(rate = 0.5)(fc1)

    final = Dense(units = num_classes,activation='sigmoid')(drop1)

    model = Model(inputs=base_model.input, outputs=final)

    return model

In [ ]:
# SA from SE on ResNet50

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    # Load pre-trained VGG16 model without top (fully connected) layers
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Get output tensor of the last VGG16 layer
    x = base_model.output

    conv1 = Conv2D(filters = 64, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(x)
    maxpool1 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv1)

    # attn1 = Attention(maxpool1, ))

    conv2 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2))(conv2)

    conv3 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool2)
    maxpool3 = MaxPooling2D(pool_size = (1, 1), strides=(2, 2))(conv3)

    attn = Attention(attention_units)([maxpool3, maxpool3])

    flattened = Flatten()(attn)

    fc1 = Dense(units = 512, activation = 'relu')(flattened)
    drop1 = Dropout(rate = 0.5)(fc1)

    final = Dense(units = num_classes,activation='sigmoid')(drop1)

    model = Model(inputs=base_model.input, outputs=final)

    return model

AttributeError: 'ImageDataGenerator' object has no attribute 'next'

In [ ]:
# SA6L on VGG16
def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    # Load pre-trained VGG16 model without top (fully connected) layers
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Get output tensor of the last VGG16 layer
    x = base_model.output

    conv1 = Conv2D(filters = 64, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(x)
    maxpool1 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding='same')(conv1)

    # attn1 = Attention(maxpool1, ))

    conv2 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding='same')(conv2)

    conv3 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool2)
    maxpool3 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding='same')(conv3)

    conv4 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool3)
    maxpool4 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding='same')(conv4)

    conv5 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool4)
    maxpool5 = MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding='same')(conv5)

    conv6 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool5)
    maxpool6 = MaxPooling2D(pool_size = (1, 1), strides=(2, 2))(conv6)

    attn = Attention(attention_units)([maxpool6, maxpool6])

    flattened = Flatten()(attn)

    fc1 = Dense(units = 512, activation = 'relu')(flattened)
    drop1 = Dropout(rate = 0.5)(fc1)

    final = Dense(units = num_classes,activation='sigmoid')(drop1)

    model = Model(inputs=base_model.input, outputs=final)

    return model

In [ ]:
# SCRAP

# Define the model
def squeeze_excite_block(input_tensor, ratio=8):
    # Squeeze operation
    squeeze = GlobalAveragePooling2D()(input_tensor)

    # Excitation operation
    excitation = Dense(input_tensor.shape[-1] // ratio, activation='relu')(squeeze)
    excitation = Dense(input_tensor.shape[-1], activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, -1))(excitation)

    # Scale the input tensor
    scaled_input = Multiply()([input_tensor, excitation])

    return scaled_input

def convolutional_block_attention_module(input_tensor, ratio=8):
    # Channel attention
    channel_avg = GlobalAveragePooling2D()(input_tensor)
    channel_max = GlobalAveragePooling2D()(input_tensor)
    channel_shared = Dense(input_tensor.shape[-1] // ratio, activation='relu', kernel_regularizer=regularizers.l2(0.01))(channel_avg) + \
                     Dense(input_tensor.shape[-1] // ratio, activation='relu', kernel_regularizer=regularizers.l2(0.01))(channel_max)
    channel_attention = Dense(input_tensor.shape[-1], activation='sigmoid', kernel_regularizer=regularizers.l2(0.01))(channel_shared)
    channel_attention = Reshape((1, 1, -1))(channel_attention)

    # Spatial attention
    spatial_avg = Reshape((1, 1, input_tensor.shape[-1]))(GlobalAveragePooling2D()(input_tensor))
    spatial_max = Reshape((1, 1, input_tensor.shape[-1]))(GlobalAveragePooling2D()(input_tensor))
    spatial_shared = Add()([Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(spatial_avg),
                           Dense(1, activation='relu', kernel_regularizer=regularizers.l2(0.01))(spatial_max)])
    spatial_attention = Activation('sigmoid')(spatial_shared)

    # Combine channel and spatial attention
    attention = Multiply()([input_tensor, channel_attention, spatial_attention])

    return attention

def build_dilated_cnn_with_attention(input_shape, num_classes, dilation_rate, attention_units):
    # Load pre-trained VGG16 model without top (fully connected) layers
    base_model = VGG19(weights='imagenet', include_top=False, input_shape=input_shape)

    # Freeze pre-trained layers
    for layer in base_model.layers:
        layer.trainable = False

    # Get output tensor of the last VGG16 layer
    x = base_model.output

    conv1 = Conv2D(filters = 64, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(x)
    conv2 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(conv1)
    # normalize = BatchNormalization()(conv2)
    conv3 = Conv2D(filters = 128, kernel_size = (3, 3), activation = 'relu', padding='same')(conv2)
    # maxpool3 = MaxPooling2D(pool_size = (2, 2), strides=(1, 1))(conv3)

    conv4 = Conv2D(filters = 256, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(conv3)
    conv5 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(conv4)
    normalize = BatchNormalization()(conv5)
    conv6 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(conv5)
    maxpool6 = MaxPooling2D(pool_size = (2, 2), strides=(1, 1))(conv6)

    conv7 = Conv2D(filters = 256, dilation_rate = dilation_rate, kernel_size = (3, 3), activation = 'relu', padding='same')(maxpool6)
    conv8 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(conv7)
    normalize = BatchNormalization()(conv8)
    conv9 = Conv2D(filters = 256, kernel_size = (3, 3), activation = 'relu', padding='same')(normalize)
    maxpool9 = MaxPooling2D(pool_size = (2, 2), strides=(1, 1))(conv9)


    attn = Attention(attention_units)([maxpool9, maxpool9])
    # attn = convolutional_block_attention_module(maxpool9)

    flattened = Flatten()(attn)

    fc1 = Dense(units = 512, activation = 'relu')(flattened)
    # drop1 = Dropout(rate = 0.5)(fc1)

    final = Dense(units = num_classes,activation='sigmoid')(fc1)

    model = Model(inputs=base_model.input, outputs=final)

    return model